In [ ]:
# For now, I'm just focusing on delta hedging. After that is done, we make a system of equations
# to set delta and vega to 0 and solve with np.linalg.solve() (see examples/ex6sol for reference).
# %pip install pyfinance
# %pip install py_vollib
# %pip install pandas
# %pip install matplotlib

import pandas as pd
from pyfinance.options import BSM
from datetime import datetime, timedelta
import py_vollib
from py_vollib.black_scholes.implied_volatility import implied_volatility
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.pyplot as plt

df = pd.read_csv("../data/apple.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df['date'] = pd.to_datetime(df['Date'])
df['T'] = (pd.to_datetime('2024-01-19') - df['date']).dt.days / 252 # T = time to expiration in years, 252 trading days
df = df.drop(["date"], axis=1)
df.head()

#interest rate data
df2 = pd.read_csv('../code/csv-data/daily-treasury-rates.csv')
df2 = df2[['Date', '13 WEEKS COUPON EQUIVALENT']]
df2['Date'] = pd.to_datetime(df2['Date'], format='%m/%d/%Y')
df2['Date'] = df2['Date'].dt.strftime('%Y-%m-%d')
df2 = df2[df2['Date'] >= '2023-08-21']
df2['13 WEEKS COUPON EQUIVALENT'] = df2['13 WEEKS COUPON EQUIVALENT'] / 100
df_result = pd.merge(df, df2, on='Date', how='left')
df_result.rename({"13 WEEKS COUPON EQUIVALENT": "Interest rate"}, axis="columns", inplace=True)
df_result['Interest rate'].fillna(df_result['Interest rate'].median(), inplace=True)
df_result.head()

# Delta-vega hedging a single option

In [ ]:
# Copy-paste spaghetti, refactor
def delta_vega_hedge(call_option_prices):
    interest_rate = call_option_prices.iloc[0]['Interest rate']
    option_type = 'call'
    stock_price = call_option_prices.iloc[0]['Underlying']
    option_price = call_option_prices.iloc[0]['C170']
    strike = 170
    time_to_maturity = call_option_prices.iloc[0]['T']
    volatility = implied_volatility(price=option_price, S=stock_price, K=strike, t=time_to_maturity, r=interest_rate, flag='c')

    # Initial stuff when t = 0
    bsm_obj = BSM(kind=option_type, S0=stock_price, K=strike, T=time_to_maturity, r=interest_rate, sigma=volatility)
    OP = bsm_obj.value() # Value of the option according to BSM

    # This needs to be different ??
    # RE = how is this with delta-vega ??
    delta = bsm_obj.delta()
    vega = bsm_obj.vega()
    
    print("Initial values:")
    print("Date:", call_option_prices.iloc[0]['Date'])
    print("Value of single call option / long position according to BSM: ", round(OP, 2))
    print("Value of the Delta of the option according to BSM: ", round(delta, 2))
    print("Value of the Vega of the option according to BSM: ", round(vega, 2))
    print("Value of the implied volatility of option: ", round(volatility, 2))
    print('\n')

    for i in range(1, len(call_option_prices)-1, 1):
        new_bsm_obj = BSM(kind=option_type, S0=call_option_prices['Underlying'][i], K=strike, T=call_option_prices['T'][i], r=call_option_prices['Interest rate'][i], sigma=volatility)
        new_OP = new_bsm_obj.value()
        
        
        # This needs to be different ??
        # new_RE = how is this with delta-vega ??
        
        d_OP = new_OP - OP

        # This also needs changing
        # d_RE = delta*(call_option_prices['Underlying'][i+1] - call_option_prices['Underlying'][i])
        # Affects also this
        # a_0 = d_OP - d_RE
        # total_mean_squared_error.append(a_0**2)
    
        print('==================================================')
        print("Date:", call_option_prices['Date'][i])
        print("Value of single call option / long position according to BSM: ", round(new_OP, 2))
        print("Value of the Delta of the option according to BSM: ", round(delta, 2))
        print("Value of the Vega of the option according to BSM: ", round(vega, 2))
        print("Value of the new long position according to BSM: ", round(new_OP))

        print("Difference of the long positions: ", round(d_OP))

        print('==================================================')
        print('\n')
    
        OP = new_OP
        
        #RE = new_RE
            
        # Change modulo to get different hedging frequencies
        if i % 2 == 0:
            delta = new_bsm_obj.delta()
            vega = new_bsm_obj.vega()
            volatility = implied_volatility(price=OP, S=call_option_prices['Underlying'][i], K=strike, t=call_option_prices['T'][i], r=call_option_prices['Interest rate'][i], flag='c')
            print("Rehedging date:", call_option_prices['Date'][i])
            print("Delta updated, new Delta is:", round(delta, 2))
            print("Vega updated, new Vega is:", round(vega, 2))
            print("Volatility updated, new implied volatility is:", round(volatility, 2))

            print('\n')

